^---- Click Me!
# **Semiconductor Material Characterization**

Authors:
- Aaron Carman
- Zachary Coon
- Clifton Dey

Takes several material properties as inputs, and outputs 2D plots showing the effects on breakdown voltage, forward voltage, intrinsic field, and depeltion width.

TODO:
1. Add support for dropdown menu to automatically populate cells with common material properties.

In [8]:
# nbi:hide_in

import matplotlib.pyplot as plt         # takes care of data visualization
import numpy as np                      # gives more math operations
import matplotlib                       # gives access to more advanced tools
from ipywidgets import interact         # gives us widgets to use
import ipywidgets as widgets
import nbinteract as nbi
from IPython.display import display
from ipywidgets import GridspecLayout

rcParams = matplotlib.rcParams          # pull current graph parameters
rcParams['font.size'] = 8.0            # set default font to 10
rcParams['axes.titlesize'] = 16.0       # change title font to 20
rcParams['font.weight'] = 'normal'      # set default font to normal
rcParams['axes.titleweight'] = 'bold'   # set title font to bold

q = 1.602e-19       # elementary charge
eps0 = 8.85e-14     # permittivity of free space (eps0)

In [9]:
# nbi:hide_in

def mainSection(dielectric_constant, T, crit_E, Me, Mh, Nc, Nv, Ni):
    eps = dielectric_constant*eps0      # material permittivity
    Ec = crit_E                         # critical E-field breakdown in V/cm

    KTq = 1.38e-23*T/q

    Na = np.logspace(15, 19, 100)   # acceptor doping that we want to sweep
    Nd = np.logspace(15, 19, 100)   # donor doping that we want to sweep

    Ncols = len(Na)         # store the number of columns
    Nrows = len(Nd)         # and the number of rows

    Voltage = np.empty((Nrows, Ncols))              # empty matrix for forward voltage
    Depletion = np.empty((Nrows, Ncols))            # empty matrix for depletion width
    BreakdownVoltage = np.empty((Nrows, Ncols))     # empty matrix for breakdown voltage
    ElectricField = np.empty((Nrows, Ncols))        # empty matrix for critical electric field

    for ii in range(0, Ncols):                      # for every element 
        for rr in range(0, Nrows):                  # in each matrix
            ln = np.log((Na[ii]*Nd[rr])/(Ni**2))    # calculate forward voltage
            Voltage[rr, ii] = KTq*ln                
            
            two_E = (2*eps)/q
            W0 = np.sqrt((two_E*Voltage[rr,ii]*((1/Na[ii]+1/Nd[rr]))))  # calculate depletion width
            Depletion[rr, ii] = W0*1e-2

            qn = q*Nd[rr]/eps
            N = (Na[ii]/(Nd[rr]*(Nd[rr]+Na[ii])))       # calculate carrier concentration
            
            ElectricField[rr,ii] = (qn*np.sqrt(two_E*N*Voltage[rr,ii]))     # calculate E-field
            
            V = (((Ec/qn)**2)/(two_E*N))
            BreakdownVoltage[rr,ii] = Voltage[rr,ii]-V          # calculate breakdown voltage


    fig, ax = plt.subplots(2,2)             # 2x2 subplot array for now
    fig.set_size_inches((12,8))             # set the size of the whole figure

    ax[0,0].set_title('Breakdown Voltage (Log Scale)')          # set title
    ax[0,0].set(xlabel='Acceptor Concentration', ylabel='Donor Concentration')  # set x and y labels
    # plot a color plot spanning the whole range we have simulated
    im = ax[0,0].imshow(np.log10(abs(BreakdownVoltage)), extent=[min(np.log10(Na)), max(np.log10(Na)), min(np.log10(Nd)), max(np.log10(Nd))], cmap='jet')
    # add a colorbar
    cbar = plt.colorbar(im, ax=ax[0,0])
    # add a label
    cbar.set_label('10^X Volts')

    # repeat the process several times
    ax[0,1].set_title('Voltage (Log Scale)')
    ax[0,1].set(xlabel='Acceptor Concentration', ylabel='Donor Concentration')
    im = ax[0,1].imshow(np.log10(abs(Voltage)), extent=[min(np.log10(Na)), max(np.log10(Na)), min(np.log10(Nd)), max(np.log10(Nd))], cmap='jet')
    cbar = plt.colorbar(im, ax=ax[0,1])
    cbar.set_label('10^X Volts')

    ax[1,0].set_title('Intrinsic Electric Field (Log Scale)')
    ax[1,0].set(xlabel='Acceptor Concentration', ylabel='Donor Concentration')
    im = ax[1,0].imshow(np.log10(abs(ElectricField)), extent=[min(np.log10(Na)), max(np.log10(Na)), min(np.log10(Nd)), max(np.log10(Nd))], cmap='jet')
    cbar = plt.colorbar(im, ax=ax[1,0])
    cbar.set_label('10^X V/cm')

    ax[1,1].set_title('Depletion Width (Log Scale)')
    ax[1,1].set(xlabel='Acceptor Concentration', ylabel='Donor Concentration')
    im = ax[1,1].imshow(np.log10(abs(Depletion)), extent=[min(np.log10(Na)), max(np.log10(Na)), min(np.log10(Nd)), max(np.log10(Nd))], cmap='jet')
    cbar = plt.colorbar(im, ax=ax[1,1])
    cbar.set_label('10^X m')

    fig.tight_layout(pad=2.0)

    plt.show()

Me = 0.18           # effective mass of electron (it's not used anywhere Clif wtf)
Mh = 1              # effective mass of hole (not used)
Nc = 1.2e18         # effective DOS of conductance band (not used)
Nv = 4.6e19         # effective DOS of valence band (not used)
Ni = 1e-10          # intrinsic carrier concentration

eps = widgets.FloatText(value = 8.9, description='Dielectric Constant')
kelvin = widgets.FloatText(value = 300, description='Temp in K')
Me = widgets.FloatText(value = 0.18, description='Electron Effective Mass')
Mh = widgets.FloatText(value = 1, description='Hole Effective Mass')
Nc = widgets.FloatText(value = 1.2e18, description='DOS (Conductance band)')
Nv = widgets.FloatText(value = 4.6e19, description='DOS (Valence band)')
Ni = widgets.FloatText(value = 1e-10, description='Intrinsic Carrier Concentration')
crit_E = widgets.FloatText(value = 3.6e6, description='Critical Electric Field')

ui = widgets.GridBox([eps,kelvin,Ni,crit_E,Me,Mh,Nc,Nv], layout=widgets.Layout(grid_template_columns="repeat(2, 400px)"))
out = widgets.interactive_output(mainSection,  {'dielectric_constant':eps, 'T':kelvin, 'crit_E':crit_E, 'Me':Me, 'Mh':Mh, 'Nc':Nc, 'Nv':Nv, 'Ni':Ni})
display(ui, out)

GridBox(children=(FloatText(value=8.9, description='Dielectric Constant'), FloatText(value=300.0, description=…

Output()